In [ ]:
import geopandas as gpd
import requests
import zipfile
import io
import matplotlib.pyplot as plt
import os
import fiona
import pandas as pd
import sys
#%matplotlib inline  # jupyter "magic" to display plots in notebook
pd.set_option('mode.chained_assignment', None)
import numpy as np
from bs4 import BeautifulSoup
import re
pd.options.display.max_columns = None
pd.options.mode.chained_assignment = None  # default='warn'
import sys
sys.path.append("../")
import src.soporteScrapping as scr

parameters

In [ ]:
url = 'https://download.geofabrik.de/europe/italy/'
country = 'IT'
country_clean = 'IT'

get links

In [ ]:
links,flattened = scr.getLinks(url)
links

In [ ]:
flattened

data conversion table input

In [ ]:
ruta = os.path.dirname(os.getcwd()) + '\\data'

In [ ]:
inPOIS = pd.ExcelFile(f'{ruta}\\POIS_Classification' + '.xlsx')
sheets_list = inPOIS.sheet_names[0]
POIS = pd.read_excel(inPOIS, sheet_name = sheets_list)
POIS.rename({'Tipo2':'Subtipo', 'Subtipo':'fclass'}, axis = 1, inplace=True)
POIS.head()

downloading shapefile

In [ ]:
print('Downloading shapefile...')
patron = r'(.*).shp.zip'
for link, flat in zip(links, flattened):
    local_path = os.path.dirname(os.getcwd()) + f'\\data\\tmpOSM_{country}'
    r = requests.get(link)
    z = zipfile.ZipFile(io.BytesIO(r.content), mode='a')
    folder = re.findall(patron, flat)
    os.mkdir(os.path.join(local_path, folder[0]))
    z.extractall(path=(local_path + '//'+ folder[0])) # extract to folder
    print(f"Done with {link}")
filenames = [y for y in sorted(z.namelist()) for starting in ['gis_osm_landuse_a_free_1', 'gis_osm_buildings_a_free_1'] if y.startswith(starting)] 
print(filenames)

POIs

In [ ]:
alldf_pois = []
for subdir, dirs, files in os.walk(local_path):
    #global alldf_landuse
    for file in files:
        #print(file)
        if file.startswith('gis_osm_pois_free_1.shp'):
            print(file, subdir)
            df_pois = gpd.read_file(subdir + '\\' + 'gis_osm_pois_free_1' + '.shp')
            alldf_pois.append(df_pois)

In [ ]:
df_pois = pd.concat([(alldf_pois[n]) for n in list(range(len(alldf_pois)))])

In [ ]:
print(df_pois.shape)
df_pois.head()

merge with equivalence table

In [ ]:
POIS.head()

In [ ]:
df_pois = df_pois.merge(POIS[['Tipo','Subtipo', 'fclass']], on = 'fclass', how = 'left')
df_pois.value_counts()

In [ ]:
df_pois.info()

In [ ]:
df_pois.head()

reproject

In [ ]:
df_pois.crs ## tiene que estar en WGS 84

In [ ]:
df_pois.to_crs("EPSG:4326")

In [ ]:
df_pois.crs ## tiene que estar en WGS 84

export

In [ ]:
local_path

In [ ]:
df_pois.to_json()  